In [3]:
# Cargamos los datos de los ficheros y los mostramos

import json


with open('peliculas_humor.json', 'r') as c:
        humor = json.load(c)
print(type(humor))


with open('peliculas_terror.json', 'r') as t:
        terror = json.load(t)
print(type(terror))


<class 'list'>
<class 'list'>


In [4]:
# Hacemos un dataframe con los datos de los dos ficheros

import pandas as pd

todopelis = comedia + terror

df = pd.DataFrame(todopelis)
df_total = df.sample(frac=1).reset_index(drop=True)
df_total

,title,genre,sinopsis
0,The Greatest Hits,"[Comedia, Drama, Fantástico]",[Una historia de amor centrada en la conexión ...
1,The Dinner,[Comedia],['The Dinner' es una obra que narra la histori...
2,The Lost Boys,"[Terror, Thriller]",[Nueva versión del largometraje de 1987 que si...
3,Score,"[Comedia, Drama]",[Un entrenador de fútbol universitario rompe t...
4,The Red,[Terror],[Obsesionada con estar a la altura del legado ...
...,...,...,...
995,Velma,"[Serie de TV, Animación, Comedia, Aventuras, I...",[Serie de TV (2023-). 1 temporada. 10 episodio...
996,The Queen Mary,"[Terror, Drama]","[El icónico transatlántico, The Queen Mary, es..."
997,The Forest Hills,"[Drama, Terror]",[Un hombre es atormentado por visiones de pesa...
998,Rockbottom,[Comedia],[Sigue a la banda de hair metal CougarSnake de...


In [33]:
from sklearn.model_selection import train_test_split
sinopsis = df_total['sinopsis']
genero = df_total['genre']
sinopsis_train, sinopsis_test, genero_train, genero_test = train_test_split(sinopsis, genero, test_size=0.4, random_state=30)

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#Creamos el vectorizador sin especificar las stopwords
vectorizador = CountVectorizer()

#Aplicamos el vectorizador a los datos de entrenamiento y prueba
sinopsis_train_vector = vectorizador.fit_transform(sinopsis_train)
sinopsis_test_vector = vectorizador.transform(sinopsis_test)

#Creamos el modelo de clasificación y lo entrenamos usando LogisticRegression
clf = LogisticRegression()
clf.fit(sinopsis_train_vector, genero_train)


preds = clf.predict(sinopsis_test_vector)
print(classification_report(genero_test,preds,target_names=['Horror','Comedia']))


AttributeError: 'list' object has no attribute 'lower'